In [5]:
import pandas as pd
import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_datareader.data as web
from pandas.tseries.offsets import BDay
import requests
import bs4
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import statsmodels.api as sm
from sklearn.cross_validation import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import matplotlib.patheffects as path_effects
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
import sys
reload(sys)
import csv
sys.setdefaultencoding('utf8')
matplotlib.style.use('ggplot')
from pandas.tseries.offsets import BDay



In [6]:
etfs = pd.read_csv('etflist.csv')
etfs = etfs.Symbol.tolist()

score vs holdingtime
score vs holdingtime (after finding optimal waitingtobuy time)
support vector


In [ ]:
df = action(etfs,2500)

In [ ]:
def action(stocklist,numdates):
    actionlist=[]
    for ticker in stocklist:
        status = ''
        period = datetime.timedelta(days=numdates)
        end = datetime.date.today()
        start = end - period
        df = web.DataReader(ticker, 'yahoo', start, end)
        del df['Close']
        df.columns = [['open','high','low','volume','close']]
        df = df.reset_index()
        df['yesterdayclose'] = df.close.shift(1)
        df['gainloss'] = df.close - df.yesterdayclose
        df['gain'] = df.gainloss.apply(lambda x: x if x > 0 else 0)
        df['loss'] = df.gainloss.apply(lambda x: -x if x < 0 else 0)
        df['avggain'] = pd.rolling_sum(df.gain, 14)/14
        df['avgloss'] = pd.rolling_sum(df.loss, 14)/14
        df['RS'] = df.avggain/df.avgloss
        df['RSI'] = 100-(100/(1+df.RS))
        df['vix'] = df.apply(lambda row: vix(row['Date']), axis=1)
        df['onewkgain'] = (df.close - df.close.shift(5))*100/df.close
        df['50MA'] = pd.rolling_mean(df.close,50)
        df['100MA'] = pd.rolling_mean(df.close,100)
        df['200MA'] = pd.rolling_mean(df.close,200)
        for row in range(len(df)):
            if df.ix[row].RSI > 80 and status == '':
                status = 'shorted'
                shortdate = df.ix[row].Date
                shortprice = df.ix[row].close 
            elif df.ix[row].RSI < 20 and status == '':
                buydate = df.ix[row].Date
                buyprice = df.ix[row].close
                status = "own"
            if status == 'own' and df.ix[row].RSI > 45:
                actionlist.append([ticker,buydate,buyprice,'regular',df.ix[row][0],df.ix[row][5],df.ix[row][6],df.ix[row][13],
                                   df.ix[row][14],df.ix[row][15],df.ix[row][16],df.ix[row][17],df.ix[row][18]])
                status = ''
            elif status == 'shorted' and df.ix[row].RSI < 45:
                actionlist.append([ticker,df.ix[row][0],df.ix[row][5],'short',shortdate,shortprice,df.ix[row][6],df.ix[row][13],
                                   df.ix[row][14],df.ix[row][15],df.ix[row][16],df.ix[row][17],df.ix[row][18]])
                status = ''
    actiondf = pd.DataFrame(columns = ['Ticker','BuyDate','BuyPrice','Type','SellDate','SellPrice','yesterdayclose',
                                       'rsi','vix','onewkgain','50MA','100MA','200MA'])
    for i in range(len(actionlist)):
        actiondf.loc[i] = [actionlist[i][0],actionlist[i][1],actionlist[i][2],actionlist[i][3],actionlist[i][4],actionlist[i][5],
                           actionlist[i][6],actionlist[i][7],actionlist[i][8],actionlist[i][9],actionlist[i][10],actionlist[i][11],
                           actionlist[i][12]]
    actiondf['daysholding'] = abs(actiondf.SellDate - actiondf.BuyDate)
    actiondf['percentgain'] = (actiondf.SellPrice - actiondf.BuyPrice)*100/actiondf.BuyPrice
    return actiondf

def vix(datey):
    datey = pd.to_datetime(datey)
    end = datetime.date(datey.year,datey.month,datey.day)
    vix = web.DataReader('^vix', 'yahoo',end,end)
    return vix['Adj Close'][0]

In [365]:
def getetfinception(ticker):
    time.sleep(5)
    url_template = 'http://etfdb.com/etf/{}'
    response = requests.get(url_template.format(ticker))
    page = response.text
    soupy = BeautifulSoup(page,'lxml')
    inceptionsoup = soupy.findAll(class_="list-unstyled")
    inceptionlist = []
    for i in inceptionsoup:
        try:
            inceptionlist.append(i.text.strip())
        except:
            pass
    a = inceptionlist[0].find('\n\n\nTax Form:')
    return int(inceptionlist[0][a-4:a])


def getetfcategory(ticker):   
    url_template = 'http://etfdb.com/etf/{}'
    response = requests.get(url_template.format(ticker))
    page = response.text
    soupy = BeautifulSoup(page,'lxml')
    categorysoup = soupy.findAll(class_="list-inline")
    categorylist = []
    for i in categorysoup:
        try:
            categorylist.append(i.text.strip())
        except:
            pass

    return categorylist[2][9:]

def performancescore(x):
    if x > 2:
        return 2
    elif x > 0: 
        return 1
    elif x > -2:
        return -1
    else: return -2


import time
def getetftype(ticker):
    time.sleep(5)
    url_template = 'http://etfdb.com/etf/{}'
    response = requests.get(url_template.format(ticker))
    page = response.text
    soupy = BeautifulSoup(page,'lxml')
    typesoup = soupy.findAll(class_="list-unstyled")
    typelist = []
    for i in typesoup:
        try:
            typelist.append(i.text.strip())
        except:
            pass
    return typelist[0][(typelist[0].find('\nStructure:\n')+12):(typelist[0].find('\n\n\nExpense Ratio:'))]


In [1]:
df['inceptionyear'] = df.apply(lambda x: getetfinception(x.Ticker),axis=1)
# df['category'] = df.apply(lambda x: getetfcategory(x.Ticker),axis=1)
# df['performancescore'] = df.percentgain.to_frame().applymap(performancescore)
# df['ETFtype'] = df.apply(lambda x: getetftype(x.Ticker),axis=1)
# short = df[df.Type=='short']
# regular = df[df.Type=='regular']

NameError: name 'df' is not defined

In [333]:
len(df[df.percentgain>0])/float(len(df))

0.6210836712126797

In [334]:
df.percentgain.mean()

0.8196064740122232

In [336]:
df.daysholding.mean()

Timedelta('26 days 13:59:41.422779')

In [ ]:
short.percentgain.mean()

In [338]:
regular.percentgain.mean()

5.457683887164806

In [340]:
len(short)

1565

In [355]:
dfhead = df.tail(25)

In [358]:
dfhead['inceptionyear'] = dfhead.apply(lambda x: getetfinception(x.Ticker),axis=1)
dfhead['category'] = dfhead.apply(lambda x: getetfcategory(x.Ticker),axis=1)
dfhead['performancescore'] = dfhead.percentgain.to_frame().applymap(performancescore)
dfhead['ETFtype'] = dfhead.apply(lambda x: getetftype(x.Ticker),axis=1)

In [361]:
df.head(25)

,Ticker,BuyDate,BuyPrice,Type,SellDate,SellPrice,yesterdayclose,rsi,vix,onewkgain,50MA,100MA,200MA,daysholding,percentgain
0,SPY,2010-01-22,95.682624,short,2010-01-08,100.378704,97.864196,42.142340,27.309999,-5.237617,97.549918,NaN,NaN,14 days,4.907976
1,SPY,2010-05-04,103.387301,short,2010-03-01,98.030664,105.876973,39.237507,23.840000,-0.816888,102.823551,100.011653,NaN,64 days,-5.181136
2,SPY,2010-07-06,90.929651,regular,2010-07-13,96.931522,95.490715,50.286090,24.559999,6.191867,96.795994,99.664776,NaN,7 days,6.600565
3,SPY,2010-08-24,93.280896,regular,2010-09-01,95.870804,93.086431,49.375466,23.889999,2.323434,95.765441,97.964799,98.244229,8 days,2.776461
4,SPY,2010-11-23,105.261769,short,2010-09-20,101.493853,106.808042,43.892508,20.629999,0.244824,104.360774,100.449738,100.086130,64 days,-3.579567
5,SPY,2011-03-01,116.964140,short,2010-12-16,110.922535,118.947341,42.832106,21.010000,-0.687397,115.501045,111.122195,104.069643,75 days,-5.165348
6,SPY,2011-04-18,117.140288,short,2011-04-05,119.544829,118.468161,39.938026,16.959999,-1.455276,117.865346,115.134377,107.792057,13 days,2.052702
7,SPY,2011-07-27,117.757108,short,2011-07-07,122.049016,120.218643,34.403715,22.980000,-1.569669,118.091084,118.417887,114.992911,20 days,3.644712
8,SPY,2011-08-08,101.220617,regular,2011-08-24,106.468292,104.989566,47.296628,35.900002,-1.346542,114.069040,116.616184,115.212987,16 days,5.184393
9,SPY,2012-04-09,126.619524,short,2012-01-10,117.776286,128.057750,37.675719,18.809999,-1.873822,125.402735,119.796161,115.427768,90 days,-6.984103


In [ ]:
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import matplotlib.patheffects as path_effects
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


def plot_confusion_matrix(cm, title='Titanic Survival Model', labels=['Survived','Deceased'], cmap=plt.cm.Blues):
    
    plt.figure(figsize=(7,7))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, labels)
    plt.yticks(tick_marks, labels)
        
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
    width, height = cm.shape
    
    for x in xrange(width):
        for y in xrange(height):
            plt.annotate(str(cm[x][y]), xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center',
                        color = 'white',
                        fontsize=18).set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'),
                                                       path_effects.Normal()])

def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test, y_pred)
    
    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)
    
    print cm
    print cr
    
    return a

all_models = {}



def getROCcurve(roc, title='ROC Curve', xlabel ='% of False Positives', ylabel = '% of True Positives'):
    
    fpr, tpr, thresholds = roc
    
    fig,ax = plt.subplots()
    ax.plot(fpr, tpr, label='ROC Curve')

    fig.set_size_inches(7, 6, forward=True)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend(loc='best')

proba = model.predict_proba(X).T[1]
roc = roc_curve(y,proba)

getROCcurve(roc, title='Predicting Survivorship', ylabel='% of Survivors Predicted Correctly',\
            xlabel='% of Deceased Predicted as Survivors')





## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
ll_models['lr'] = {'model': gsknn.best_estimator_,
                     'score': evaluate_model(KNeighborsClassifier())}

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.grid_search import GridSearchCV

params = {'n_neighbors': range(2,60)}
gsknn = GridSearchCV(KNeighborsClassifier(),params, n_jobs=-1,cv=KFold(len(y), n_folds=3, shuffle=True))
gsknn.fit(X, y)
gsknn.best_params_
gsknn.best_score_
all_models['knn'] = {'model': gsknn.best_estimator_,
                     'score': evaluate_model(KNeighborsClassifier())}


baggingknn = BaggingClassifier(KNeighborsClassifier())
evaluate_model(baggingknn)
bagging_params = {'n_estimators': [10, 20],
                  'max_samples': [0.7, 1.0],
                  'max_features': [0.7, 1.0],
                  'bootstrap_features': [True, False]}


gsbaggingknn = GridSearchCV(baggingknn,
                            bagging_params, n_jobs=-1,
                            cv=KFold(len(y), n_folds=3, shuffle=True))

# gsbaggingknn.fit(X, y)
# gsbaggingknn.best_params_
all_models['gsbaggingknn'] = {'model': gsbaggingknn.best_estimator_,
                              'score': evaluate_model(gsbaggingknn.best_estimator_)}

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
all_models['dt'] = {'model': dt,
                    'score': evaluate_model(dt)}
params = {'criterion': ['gini', 'entropy'],
          'splitter': ['best', 'random'],
          'max_depth': [None, 5, 10],
          'min_samples_split': [2, 5],
          'min_samples_leaf': [1, 2, 3]}

gsdt = GridSearchCV(dt,
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

# gsdt.fit(X, y)
# print gsdt.best_params_
# print gsdt.best_score_

all_models['gsdt'] = {'model': gsdt.best_estimator_,
                      'score': evaluate_model(gsdt.best_estimator_)}

gsbaggingdt = GridSearchCV(BaggingClassifier(gsdt.best_estimator_),
                           bagging_params, n_jobs=-1,
                           cv=KFold(len(y), n_folds=3, shuffle=True))

# gsbaggingdt.fit(X, y)

# print gsbaggingdt.best_params_
# print gsbaggingdt.best_score_

all_models['gsbaggingdt'] = {'model': gsbaggingdt.best_estimator_,
                             'score': evaluate_model(gsbaggingdt.best_estimator_)}

## Support Vector Machines

In [ ]:
from sklearn.svm import SVC
svm = SVC()
all_models['svm'] = {'model': svm,
                     'score': evaluate_model(svm)}
params = {'C': [0.01, 0.1, 1.0, 10.0, 30.0, 100.0],
          'gamma': ['auto', 0.1, 1.0, 10.0],
          'kernel': ['linear', 'rbf']}


gssvm = GridSearchCV(svm,
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

# gssvm.fit(X, y)
# print gssvm.best_params_
# print gssvm.best_score_

all_models['gssvm'] = {'model': gssvm.best_estimator_,
                      'score': evaluate_model(gssvm.best_estimator_)}

gsbaggingsvm = GridSearchCV(BaggingClassifier(gssvm.best_estimator_),
                           bagging_params, n_jobs=-1,
                           cv=KFold(len(y), n_folds=3, shuffle=True))

# gsbaggingsvm.fit(X, y)

# print gsbaggingsvm.best_params_
# print gsbaggingsvm.best_score_

all_models['gsbaggingsvm'] = {'model': gsbaggingsvm.best_estimator_,
                             'score': evaluate_model(gsbaggingsvm.best_estimator_)}

## Random Forest and Extra Trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

rf = RandomForestClassifier()
all_models['rf'] = {'model': rf,
                    'score': evaluate_model(rf)}



et = ExtraTreesClassifier()
all_models['et'] = {'model': et,
                    'score': evaluate_model(et)}

params = {'n_estimators':[3, 5, 10, 50],
          'criterion': ['gini', 'entropy'],
          'max_depth': [None, 3, 5],
          'min_samples_split': [2,5],
          'class_weight':[None, 'balanced']}


gsrf = GridSearchCV(RandomForestClassifier(n_jobs=-1),
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

# gsrf.fit(X, y)
# print gsrf.best_params_
# print gsrf.best_score_

all_models['gsrf'] = {'model': gsrf.best_estimator_,
                      'score': evaluate_model(gsrf.best_estimator_)}


gset = GridSearchCV(RandomForestClassifier(n_jobs=-1),
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

# gset.fit(X, y)
# print gset.best_params_
# print gset.best_score_

all_models['gset'] = {'model': gset.best_estimator_,
                      'score': evaluate_model(gset.best_estimator_)}

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier()
all_models['ab'] = {'model': ab,
                    'score': evaluate_model(ab)}

gsab = GridSearchCV(AdaBoostClassifier(n_jobs=-1),
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

# gsab.fit(X, y)
# print gsab.best_params_
# print gsab.best_score_

all_models['gsab'] = {'model': gsab.best_estimator_,
                      'score': evaluate_model(gsab.best_estimator_)}

## Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostClassifier
gb = GradientBoostClassifier()
all_models['gb'] = {'model': gb,
                    'score': evaluate_model(gb)}

gsgb = GridSearchCV(GradientBoostClassifier(n_jobs=-1),
                    params, n_jobs=-1,
                    cv=KFold(len(y), n_folds=3, shuffle=True))

# gsgb.fit(X, y)
# print gsgb.best_params_
# print gsgb.best_score_

all_models['gsgb'] = {'model': gsgb.best_estimator_,
                      'score': evaluate_model(gsgb.best_estimator_)}

## Scoring

In [ ]:
scores = pd.DataFrame([(k, v['score']) for k, v in all_models.iteritems()],
             columns=['model', 'score']).set_index('model').sort_values('score', ascending=False)

scores.plot(kind='bar')
plt.ylim(0.6, 1.1)

In [ ]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold


def retest(model):
    scores = cross_val_score(model, X, y,
                             cv=StratifiedKFold(y, shuffle=True),
                             n_jobs=-1)
    m = scores.mean()
    s = scores.std()
    
    return m, s

for k, v in all_models.iteritems():
    cvres = retest(v['model'])
    print k, 
    all_models[k]['cvres'] = cvres

In [ ]:
cvscores = pd.DataFrame([(k, v['cvres'][0], v['cvres'][1] ) for k, v in all_models.iteritems()],
                        columns=['model', 'score', 'error']).set_index('model').sort_values('score', ascending=False)

fig, ax = plt.subplots()
rects1 = ax.bar(range(len(cvscores)), cvscores.score,
                yerr=cvscores.error,
                tick_label=cvscores.index)

ax.set_ylabel('Scores')
plt.xticks(rotation=70)
plt.ylim(0.6, 1.1)

cvscores